In [1]:
import numpy as np

In [5]:
sentences = ["i like dog", "i love coffee", "i hate milk"]

In [6]:
word_list = " ".join(sentences).split()
vocab = list(set(word_list))
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for i, w in enumerate(vocab)}
n_class = len(vocab)

In [20]:
n_class

7

In [154]:
n_step = 3
batch_size = 1
n_class = len(unique_token_index)
n_hidden = 5

In [10]:
text = open('pizza.txt').read().lower()

In [88]:
text[:300]

'pizza, the delectable and iconic dish that has transcended borders and captivated taste buds worldwide, is a testament to the extraordinary fusion of flavors, creativity, and cultural significance. originating from the sun-kissed lands of italy, pizza has evolved into an art form that unites people '

In [11]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+')

tokens = tokenizer.tokenize(text)

In [91]:
unique_tokens = np.unique(tokens)
unique_token_index = {token:idx for idx, token in enumerate(unique_tokens)}

In [92]:
# потом удалить
unique_token_index

{'19th': 0,
 '20th': 1,
 'a': 2,
 'ability': 3,
 'acceptance': 4,
 'accessible': 5,
 'accommodating': 6,
 'adapt': 7,
 'adaptability': 8,
 'addition': 9,
 'adds': 10,
 'adore': 11,
 'adorned': 12,
 'advancements': 13,
 'advent': 14,
 'air': 15,
 'alike': 16,
 'all': 17,
 'allowing': 18,
 'allure': 19,
 'alone': 20,
 'also': 21,
 'alternative': 22,
 'alternatives': 23,
 'american': 24,
 'an': 25,
 'ancient': 26,
 'and': 27,
 'anticipation': 28,
 'any': 29,
 'appeal': 30,
 'apps': 31,
 'are': 32,
 'areas': 33,
 'arenas': 34,
 'aroma': 35,
 'aromatic': 36,
 'around': 37,
 'art': 38,
 'artisanal': 39,
 'artistry': 40,
 'artwork': 41,
 'as': 42,
 'asian': 43,
 'aspects': 44,
 'assimilated': 45,
 'at': 46,
 'atmosphere': 47,
 'augmented': 48,
 'automation': 49,
 'availability': 50,
 'available': 51,
 'away': 52,
 'back': 53,
 'backgrounds': 54,
 'baked': 55,
 'bakes': 56,
 'balance': 57,
 'based': 58,
 'basil': 59,
 'beautiful': 60,
 'become': 61,
 'becomes': 62,
 'becoming': 63,
 'been': 64

In [77]:
def sentences(step):
    input_words = []
    next_words = []
    for i in range(len(tokens) - step):
        input_words.append(tokens[i:i + step])
        next_words.append(tokens[i + step])

    return input_words, next_words

In [81]:
input_words, next_words = sentences(n_step)

In [84]:
input_words[:5]

[['pizza', 'the', 'delectable'],
 ['the', 'delectable', 'and'],
 ['delectable', 'and', 'iconic'],
 ['and', 'iconic', 'dish'],
 ['iconic', 'dish', 'that']]

In [85]:
next_words[:5]

['and', 'iconic', 'dish', 'that', 'has']

In [160]:
def make_data(input_words, next_words, n_class=n_class):
    input_batch = []
    target_batch = []
    eye = np.eye(n_class, dtype='float32')
    for i, j in enumerate(input_words):
        input = [unique_token_index[n] for n in j]
        target = unique_token_index[next_words[i]]
        
        input_batch.append(eye[input])
        target_batch.append(eye[target])

    return np.array(input_batch), np.array(target_batch)

In [161]:
input_batch, target_batch = make_data(input_words, next_words)   

In [162]:
import torch

input_tensor, target_tensor = torch.from_numpy(input_batch), torch.from_numpy(target_batch)

In [163]:
input_tensor.shape

torch.Size([1711, 3, 683])

In [164]:
target_tensor.shape

torch.Size([1711, 683])

In [165]:
import torch.utils.data as Data

dataset = Data.TensorDataset(input_tensor, target_tensor)

In [166]:
loader = Data.DataLoader(dataset, batch_size, True)

In [167]:
import torch.nn as nn

class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        # self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)
        self.rnn = nn.GRU(input_size=n_class, hidden_size=n_hidden)

        # fc
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, hidden, X):
        # X: [batch_size, n_step, n_class]
        X = X.transpose(0, 1) # X : [n_step, batch_size, n_class]
        out, hidden = self.rnn(X, hidden)
        # out : [n_step, batch_size, num_directions(=1) * n_hidden]
        # hidden : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        out = out[-1] # [batch_size, num_directions(=1) * n_hidden] 
        model = self.fc(out)
        return model

In [168]:
import torch.optim as optim

model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [171]:
for epoch in range(50):
    for x, y in loader:
        # hidden : [num_layers * num_directions, batch, hidden_size]
        hidden = torch.zeros(1, x.shape[0], n_hidden)
        
        # x : [batch_size, n_step, n_class]
        pred = model(hidden, x)
        
        # pred : [batch_size, n_class], y : [batch_size] (LongTensor, not one-hot)
        loss = criterion(pred, y)
        if (epoch + 1) % 10 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 0010 cost = 1.613848
Epoch: 0010 cost = 3.360435
Epoch: 0010 cost = 4.224692
Epoch: 0010 cost = 5.935943
Epoch: 0010 cost = 5.305422
Epoch: 0010 cost = 5.920528
Epoch: 0010 cost = 0.559576
Epoch: 0010 cost = 4.987471
Epoch: 0010 cost = 0.601611
Epoch: 0010 cost = 5.082021
Epoch: 0010 cost = 5.843510
Epoch: 0010 cost = 4.376613
Epoch: 0010 cost = 4.350991
Epoch: 0010 cost = 0.701353
Epoch: 0010 cost = 3.594069
Epoch: 0010 cost = 0.385177
Epoch: 0010 cost = 2.586387
Epoch: 0010 cost = 5.596378
Epoch: 0010 cost = 2.493890
Epoch: 0010 cost = 3.969493
Epoch: 0010 cost = 5.179910
Epoch: 0010 cost = 5.642361
Epoch: 0010 cost = 4.934370
Epoch: 0010 cost = 4.678754
Epoch: 0010 cost = 0.961495
Epoch: 0010 cost = 3.800025
Epoch: 0010 cost = 3.344205
Epoch: 0010 cost = 0.563545
Epoch: 0010 cost = 5.161879
Epoch: 0010 cost = 6.871839
Epoch: 0010 cost = 3.614178
Epoch: 0010 cost = 1.384909
Epoch: 0010 cost = 1.856547
Epoch: 0010 cost = 0.457087
Epoch: 0010 cost = 3.185340
Epoch: 0010 cost = 0

In [213]:
input = input_words

hidden = torch.zeros(1, len(input), n_hidden)
predict = model(hidden, input_tensor).data.max(1, keepdim=True)[1]

In [193]:
predict

tensor([[ 27],
        [156],
        [329],
        ...,
        [675],
        [598],
        [158]])

In [183]:
idx2word = {idx:token for idx, token in enumerate(unique_tokens)}
idx2word[27]

'and'

In [204]:
len(input)

1711

In [205]:
input_tensor[].shape

torch.Size([1711, 3, 683])

In [209]:
[input_words[0]]

[['pizza', 'the', 'delectable']]